# Date and time

Working with dates and times is quite common when using SQL. Since date columns have many specific features, there is a dedicated section for handling them.

In this notebook, we will work with multiple database systems: Postgres and ClickHouse. The following cell creates all the necessary Docker containers.

In [1]:
docker run --rm -d\
    -e POSTGRES_PASSWORD=docker_app \
    --name datetime_postgres_examples \
    postgres:15.4 &> /dev/null
docker run --rm -d \
    --name datetime_click_examples \
    clickhouse/clickhouse-server &> /dev/null
sleep 5

**Note**: Don't forget to stop the containers after all.

In [ ]:
docker stop datetime_postgres_examples datetime_click_examples

## Date data types

There are several data types that allow you to describe and operate with date and time. In the case of Postgres, some of them are outlined in the following table.

| Data Type                     | Description                                                               | Example                          |
|-------------------------------|---------------------------------------------------------------------------|----------------------------------|
| **DATE**                      | Stores the date (year, month, day) without time.                       | `'2024-07-03'`                  |
| **TIME**                      | Stores the time of day (hours, minutes, seconds) without time zone.    | `'14:30:57.902629'`             |
| **TIME WITH TIME ZONE (TIMETZ)** | Stores the time of day including the time zone.                        | `'14:30:57.902629+02:00'`       |
| **TIMESTAMP**                 | Stores both date and time without time zone.                            | `'2024-07-03 14:30:57.902629'`  |
| **TIMESTAMP WITH TIME ZONE (TIMESTAMPTZ)** | Stores both date and time including the time zone.        | `'2024-07-03 14:30:57.902629+02:00'` |
| **INTERVAL**                  | Represents a time span or duration (days, hours, minutes, seconds).    | `'1 day 2 hours 3 minutes'`     |


And the same for clickhouse.

| Data Type         | Description                                               | Example                |
|-------------------|-----------------------------------------------------------|------------------------|
| `Date`            | Represents a date without time (YYYY-MM-DD format).      | `2024-09-27`           |
| `DateTime`        | Represents a date and time (YYYY-MM-DD HH:MM:SS format). | `2024-09-27 14:30:00`  |
| `DateTime64`      | Represents a date and time with fractional seconds.      | `2024-09-27 14:30:00.123456` |
| `DateTime32`      | Similar to `DateTime`, but limited to 32-bit precision.  | `2024-09-27 14:30:00`  |
| `DateTime('timezone')` | `DateTime` with a specified timezone.              | `2024-09-27 14:30:00` (UTC) |
| `Time`            | Represents time (HH:MM:SS format) without date.         | `14:30:00`             |
| `Interval`        | Represents a time interval (e.g., days, hours).         | `INTERVAL 1 DAY`       |


### Date

All database systems have some types that describe jsut a date - it's a date without time.


---

As an example, consider how the `DATE` data type can be defined in Postgres. In the following cell, the types of the results from different ways of creating a date are printed.

In [18]:
docker exec -i datetime_postgres_examples psql -U postgres -d postgres << EOF
SELECT 
    pg_typeof('2999-08-12') one,
    pg_typeof(CAST('2999-08-12' AS DATE)) two,
    pg_typeof('2999-08-12'::DATE) three,
    pg_typeof(to_date('2999-08-12', 'YYYY-MM-DD')) four
;
EOF

   one   | two  | three | four 
---------+------+-------+------
 unknown | date | date  | date
(1 row)



In ClickHouse, dates are stored as strings. By using the `toDate` function, you can make ClickHouse interpret them as datetime columns.

In [10]:
docker exec -it datetime_click_examples clickhouse-client -q "
SELECT 
    toTypeName('2999-12-23'),
    toTypeName(toDate('2999-12-23')) 
FORMAT Pretty;
"

   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
   ┃ toTypeName('2999-12-23') ┃ toTypeName(toDate('2999-12-23')) ┃
   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
1. │ String                   │ Date                             │
   └──────────────────────────┴──────────────────────────────────┘


### Datetime/Timestamp

The `Datetime` type is used to describe date and time with precision.

---

The following cell demonstrates a comparison of typecasting to `DATE` and `TIMESTAMP` in postgres.

In [9]:
docker exec -i datetime_postgres_examples psql -U postgres -d postgres << EOF
SELECT 
    '2024-07-03 14:30:57.902629'::DATE,
    '2024-07-03 14:30:57.902629'::TIMESTAMP;
EOF

    date    |         timestamp          
------------+----------------------------
 2024-07-03 | 2024-07-03 14:30:57.902629
(1 row)



So in case `DATE` there is only infomation about day but in `TMESTAMP` case we're getting accuracy up to microsecond.

The same example applies to ClickHouse.

In [20]:
docker exec -it datetime_click_examples clickhouse-client -q "
SELECT 
    toDate('2024-07-03 14:30:57'),
    toDateTime64('2024-07-03 14:30:57.902629', 6)
FORMAT Pretty;
"

   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
   ┃ toDate('2024-07-03 14:30:57') ┃ toDateTime64('2024-07-03 14:30:57.902629', 6) ┃
   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
1. │                    2024-07-03 │                    2024-07-03 14:30:57.902629 │
   └───────────────────────────────┴───────────────────────────────────────────────┘
